In [3]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from scipy.ndimage import median_filter
import scipy as sp
from scipy.optimize import curve_fit

In [2]:
def openlc(filename):
    hdul = fits.open(filename)
    data = hdul[1].data
    hdul.close()
    return data

In [4]:
def filter_and_detrend(filename, start, end, polyorder):
    data = openlc(filename)
    south_atlantic_start = np.where(data['RATE'] == 0)[0][0]
    south_atlantic_end = np.where(data['RATE'] == 0)[0][-1]
    if end<south_atlantic_start:
        data['RATE'][start:end] -= (np.mean([data['RATE'][:start]]) + np.mean(data['RATE'][end:south_atlantic_start] + np.mean(data['RATE'][south_atlantic_end:])))/3
        data['RATE'][:start] -= savgol_filter(data['RATE'][:start], 10, polyorder)
        data['RATE'][end:south_atlantic_start] -= savgol_filter(data[end:south_atlantic_start], 10, polyorder)
        data['RATE'][south_atlantic_end:] -= savgol_filter(data[south_atlantic_end:], 10, polyorder)
    elif start>south_atlantic_end:
        data['RATE'][start:end] -= (np.mean([data['RATE'][:south_atlantic_start]]) + np.mean(data['RATE'][south_atlantic_end:start] + np.mean(data['RATE'][end:])))/3
        data['RATE'][:south_atlantic_start] -= savgol_filter(data['RATE'][:south_atlantic_start], 10, polyorder)
        data['RATE'][south_atlantic_end:start] -= savgol_filter(data['RATE'][south_atlantic_end:start], 10, polyorder)
        data['RATE'][end:] -= savgol_filter(data['RATE'][end:], 10, polyorder)
    else:
        print('Inputted start and end times are not valid')
    return data, south_atlantic_start, south_atlantic_end

In [7]:
def snr_rms(filename, start, end, polyorder):
    data, south_atlantic_start, south_atlantic_end = filter_and_detrend(filename, start, end, polyorder)
    if end<south_atlantic_start:
        rms = np.sqrt(np.mean(data['RATE'][:start]**2))   #ignoring the noise after the GRB
        signal = max(data['RATE'][start:end])
        snr = signal/rms
    elif start>south_atlantic_end:
        rms = np.sqrt(np.mean(data['RATE'][:south_atlantic_start]**2+data['RATE'][south_atlantic_end:start]**2))
        signal = max(data['RATE'][start:end])
        snr = signal/rms
    else:
        print('Inputted start and end times are not valid')
        snr = 0
    return snr

In [8]:
def snr_abs(filename, start, end, polyorder):
    data, south_atlantic_start, south_atlantic_end = filter_and_detrend(filename, start, end, polyorder)
    if end<south_atlantic_start:
        abs = np.mean(np.abs(data['RATE'][:start]))
        signal = max(data['RATE'][start:end])
        snr = signal/abs
    elif start>south_atlantic_end:
        abs = (np.mean(np.abs(data['RATE'][:south_atlantic_start]))+np.mean(np.abs(data['RATE'][south_atlantic_end:start])))/2
        signal = max(data['RATE'][start:end])
        snr = signal/abs
    else:
        print('Inputted start and end times are not valid')
        snr = 0
    return snr

In [9]:
def gaussian(x, A, m, s, c):
    return A*np.exp(-(x-m)**2/(2*s**2)) + c

In [11]:
def snr_gauss(filename, start, end, polyorder):
    data, south_atlantic_start, south_atlantic_end = filter_and_detrend(filename, start, end, polyorder)
    if end<south_atlantic_start:
        n, bins = np.histogram(data['RATE'][:start], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][:start]), np.std(data['RATE'][:start]), 0])
        gauss1 = popt[1]+3*popt[2]
        n, bins = np.histogram(data['RATE'][end:south_atlantic_start], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][end:south_atlantic_start]), np.std(data['RATE'][end:south_atlantic_start]), 0])
        gauss2 = popt[1]+3*popt[2]
        n, bins = np.histogram(data['RATE'][south_atlantic_end:], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][south_atlantic_end:]), np.std(data['RATE'][south_atlantic_end:]), 0])
        gauss3 = popt[1]+3*popt[2]
        gauss = (gauss1+gauss2+gauss3)/3
        signal = max(data['RATE'][start:end])
        snr = signal/gauss
    elif start>south_atlantic_end:
        n, bins = np.histogram(data['RATE'][:south_atlantic_start], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][:south_atlantic_start]), np.std(data['RATE'][:south_atlantic_start]), 0])
        gauss1 = popt[1]+3*popt[2]
        n, bins = np.histogram(data['RATE'][south_atlantic_end:start], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][south_atlantic_end:start]), np.std(data['RATE'][south_atlantic_end:start]), 0])
        gauss2 = popt[1]+3*popt[2]
        n, bins = np.histogram(data['RATE'][end:], bins=80)
        bin_center = np.array([0.5*(bins[i]+bins[i+1]) for i in range(len(bins)-1)])
        popt, pcov = curve_fit(gaussian, bin_center, n, p0=[max(n), np.mean(data['RATE'][end:]), np.std(data['RATE'][end:]), 0])
        gauss3 = popt[1]+3*popt[2]
        gauss = (gauss1+gauss2+gauss3)/3
        signal = max(data['RATE'][start:end])
        snr = signal/gauss
    else:
        print('Inputted start and end times are not valid')
        snr = 0
    return snr

In [ ]:
def snr_counts(filename, start, end, polyorder):
    data, south_atlantic_start, south_atlantic_end = filter_and_detrend(filename, start, end, polyorder)
    duration_lc = data['TIME'][-1]-data['TIME'][0]
    duration_burst = data['TIME'][end]-data['TIME'][start]
    duration_sao = data['TIME'][south_atlantic_end]-data['TIME'][south_atlantic_start]
    duration_noise = duration_lc-duration_burst-duration_sao
    if end<south_atlantic_start:
        signal = np.sum(data['RATE'][start:end])/duration_burst
        noise = np.sum(data['RATE'][:start])/duration_noise + np.sum(data['RATE'][end:south_atlantic_start])/duration_noise + np.sum(data['RATE'][south_atlantic_end:])/duration_noise
        snr = signal/noise
    elif start>south_atlantic_end:
        signal = np.sum(data['RATE'][start:end])/duration_burst
        noise = np.sum(data['RATE'][:south_atlantic_start])/duration_noise + np.sum(data['RATE'][south_atlantic_end:start])/duration_noise + np.sum(data['RATE'][end:])/duration_noise
        snr = signal/noise
    else:
        print('Inputted start and end times are not valid')
        snr = 0
    return snr